# learning PostgreSQL basic usage

In this post we see how to use a PostrgreSQL database with Python.

I decided to use the **dockerized version of PostgreSQL**. I already have Docker installed on my PC. In case you need to install it, head over to [Docker](https://docs.docker.com/docker-for-windows/install/) for Windows Docker desktop version

**1. Passo** Scarichiamo l'immagine docker di postgreSQL

`docker pull postgres`

**2. Passo** lanciamo l'immagine appena scaricata

`docker run --name test-db -e POSTGRES_PASSWORD=my_secret_password -d -p 5432:5432 postgres`

Alcune spiegazioni:

**--name** è il nome che vogliamo assegnare al container (da non confondere con **container ID**)

**-p** is the port that is going to be open in the guest machine, port 5432 of the guest machine is forwarded to port 5432 of the host. Therefore I can connect to the guest just with **localhost:5432**

To run a bash console in the guest machine use this command:
`docker exec -it <CONTAINER ID> bash`

To get all the \<CONTAINER IDs> use the command:
`docker container ls`

In the bash run the psql application: `psql -U postgres`
    
Once you get the psgl prompt, you can use different commands:
    - to list the database available: `\l`
    - to list existing user: `\du`
    - to list tables (also called relations): `\dt`


In [1]:
import os
os.system('docker run --name test-db -e POSTGRES_PASSWORD=my_secret_password -d -p 5432:5432 postgres')

125


If the container exists already, we just need to start it:

In [2]:
os.system('docker container start test-db')

0

Usiamo il pacchetto di python **psycopg2** per comunicare con il nostro database PostgreSQL. Si può installare con `pip install psycopg2` 

In [3]:
import psycopg2

Create a connection with the database, using the database name and user I got with psql commands seen above.

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=my_secret_password")

PostgreSQL needs a commit command to be issued in order to save the transaction in the database. Here we are going to set **autocommit=True**, so every transaction will be autocommitted, without the need of an exciplit commit. 

In [5]:
conn.autocommit = True

In alternativ one needs to commit any transaction with `conn.commit()`.


Get a cursor for the database.

In [6]:
cur = conn.cursor()

Create a table with the two fields, user_name and age

In [7]:
cur.execute("""
CREATE TABLE IF NOT EXISTS events_staging (
    event_it BIGINT IDENTITY(0,1),
    artist    VARCHAR,
    auth      VARCHAR,
    firstName VARCHAR,
    gender    VARCHAR,
    itemInSession INT,
    lastName  VARCHAR,
    length    FLOAT,
    level     VARCHAR,
    location  VARCHAR,
    method    VARCHAR,
    page      VARCHAR,
    registration FLOAT,
    sessionId INT,
    song      VARCHAR,
    status    INT,
    ts        NUMERIC,
    userAgent VARCHAR,
    userId    BIGINT)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS songs_staging (
    num_song    BIGINT,
    artist_id   VARCHAR NOT NULL,
    artist_latitude  VARCHAR,
    artist_longitude VARCHAR,
    artist_location  VARCHAR,
    artist_name VARCHAR NOT NULL,
    song_id     VARCHAR NOT NULL,
    start_time  NUMERIC NOT NULL,
    title       VARCHAR NOT NULL,
    duration    FLOAT,
    year        INT)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS songplays (
    songplay_id SERIAL PRIMARY KEY,
    start_time  NUMERIC NOT NULL,
    user_id     INT NOT NULL,
    level       VARCHAR,
    song_id     VARCHAR NOT NULL,
    artist_id   VARCHAR NOT NULL,
    session_id  VARCHAR NOT NULL,
    location    VARCHAR,
    user_agent  VARCHAR 
)
""")


cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    user_id iNT PRIMARY KEY, 
    first_name VARCHAR NOT NULL, 
    last_name VARCHAR NOT NULL,
    gender VARCHAR, 
    level VARCHAR
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS songs (
    song_id VARCHAR, 
    title VARCHAR NOT NULL, 
    artist_id VARCHAR, 
    year INT, 
    duration FLOAT,
    PRIMARY KEY (song_id, artist_id)
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS artists (
    artist_id VARCHAR PRIMARY KEY, 
    name VARCHAR NOT NULL, 
    location VARCHAR, 
    latitude FLOAT, 
    longitude FLOAT

)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS time (
    start_time NUMERIC PRIMARY KEY, 
    hour INT NOT NULL, 
    day INT NOT NULL, 
    week INT NOT NULL, 
    month INT NOT NULL,
    year INT NOT NULL, 
    weekday INT NOT NULL
)
""")

In [8]:
import pandas as pd
filepath='C:\\Users\\Anto\\udacity-dend\\song_data\\A\\A\\A\\TRAAAAK128F9318786.json'
df_song = pd.read_json(filepath, lines=True)

In [9]:
filepath = 'C:\\Users\\Anto\\udacity-dend\\log_data\\2018\\11\\2018-11-01-events.json'
df_log = pd.read_json(filepath, lines=True)

In [10]:
df_log.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39


In [11]:
df_log['itemInSession'] = df_log['itemInSession'].astype('int')

In [12]:
df_log.loc[0,['artist','auth','firstName','gender']].values.tolist()

[None, 'Logged In', 'Walter', 'M']

In [13]:
df_log.dtypes

artist            object
auth              object
firstName         object
gender            object
itemInSession      int32
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration       int64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId             int64
dtype: object

In [49]:
cur.execute("DROP TABLE events_staging2")

In [50]:
cur.execute("""
CREATE TABLE IF NOT EXISTS events_staging2 (
    artist    VARCHAR,
    auth      VARCHAR,
    firstName VARCHAR,
    gender    VARCHAR,
    itemInSession INT
    )
""")

In [51]:
df_log.loc[0,['artist','auth','firstName','gender','itemInSession']].tolist() 

[None, 'Logged In', 'Walter', 'M', 0]

In [52]:
df_log[['artist','auth','firstName','gender','itemInSession']].dtypes

artist           object
auth             object
firstName        object
gender           object
itemInSession     int32
dtype: object

In [53]:
%config Completer.use_jedi = False



In [17]:
df_log

AttributeError: 'DataFrame' object has no attribute 'limit'

In [19]:
df_log.iterrows()

<generator object DataFrame.iterrows at 0x000001C2DFBDC3C0>

In [55]:
for i, row in df_log[['artist','auth','firstName','gender','itemInSession']].loc[:10].iterrows():
    cur.execute("""
    INSERT INTO events_staging2 (
        artist, auth, firstName, gender, itemInSession)
    VALUES (%s,%s,%s,%s,%s)
    """, row)

In [60]:
for i, row in df_log.loc[:10].iterrows():
    cur.execute("""
    INSERT INTO events_staging (
        artist, auth, firstName, gender, itemInSession, 
        lastName, length, level, location, method,
        page, registration, sessionId, song, status,
        ts, userAgent, userId)
    VALUES (%s,%s,%s,%s,%s,
            %s,%s,%s,%s,%s,
            %s,%s,%s,%s,%s,
            %s,%s,%s)
    """, row)

In [33]:
cur.execute("INSERT INTO user_table (user_name, age) VALUES ('Luci', 666);")
cur.execute("INSERT INTO user_table (user_name, age) VALUES ('Jesus', 33);")

In [61]:
cur.execute("SELECT * FROM events_staging")
print(cur.fetchall())

[(None, 'Logged In', 'Walter', 'M', 0, 'Frye', nan, 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', 1540919166796.0, 38, None, 200, Decimal('1541105830796'), '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', 39), (None, 'Logged In', 'Kaylee', 'F', 0, 'Summers', nan, 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', 1540344794796.0, 139, None, 200, Decimal('1541106106796'), '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"', 8), ("Des'ree", 'Logged In', 'Kaylee', 'F', 1, 'Summers', 246.30812, 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', 1540344794796.0, 139, 'You Gotta Be', 200, Decimal('1541106106796'), '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"', 8), (None, 'Logged In', 'Kaylee', 'F', 2, 'Summers', nan, 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'U

In [35]:
cur.execute("INSERT INTO user_table (user_name, age) VALUES ('Maddy', 13);")

In [36]:
cur.execute("SELECT * FROM user_table")
print(cur.fetchall())

[('Luci', 666), ('Jesus', 33), ('Maddy', 13)]


In [37]:
cur.execute("SELECT * FROM user_table")
for record in cur:
    print(record)

('Luci', 666)
('Jesus', 33)
('Maddy', 13)


### Other useful commands

To know the last query issued:

In [14]:
cur.query

b'SELECT * FROM user_table'

To fetch a given number of records:

In [15]:
cur.execute("SELECT * from user_table")
cur.fetchmany(2)

[('Luci', 666), ('Jesus', 33)]

If I fetch another element it start after the already fetched records

In [16]:
cur.fetchmany(1)

[('Maddy', 13)]

If you want to delete the table use **drop**

In [17]:
cur.execute('DROP TABLE user_table')

At the end, close the cursor and the connection

In [38]:
cur.close()
conn.close()